# Data preparation
## Read and merge the sourcefiles (.xlsx) from www.artportalen.se


In [2]:
import pandas as pd
from datetime import date

### Read the files to individual data frames

In [3]:
df1 = pd.read_excel("Excelexport_1.xlsx",header=[2], index_col =[0])
df2 = pd.read_excel("Excelexport_2.xlsx",header=[2], index_col =[0])
df3 = pd.read_excel("Excelexport_3.xlsx",header=[2], index_col =[0])
df4 = pd.read_excel("Excelexport_4.xlsx",header=[2], index_col =[0])
df5 = pd.read_excel("Excelexport_5.xlsx",header=[2], index_col =[0])


### Merge the data frames together

In [4]:
#Union the dfs into one large dataframe for further processing
frames = [df2,df3,df4,df5]
merged_dfs = df1.append(frames)


In [4]:
#Check the union result, are all expected rows present?
print(len(merged_dfs))
merged_dfs.head(2)

40108


,Taxonsorteringsordning,Valideringsstatus,Rödlistade,Artnamn,Vetenskapligt namn,Auktor,Antal,Enhet,Ålder/stadium,Kön,...,Samlingsbeskrivning,Artbestämd av,Bestämningsår,Bekräftad av,Bekräftelseår,Redigeringsansvarig,Rapportör,Observatörer,DNA sekvens,Projektnamn
Id,,,,,,,,,,,,,,,,,,,,,
17364564,55023,Ovaliderad,NaN,Ringtrast,Turdus torquatus,"Linnaeus, 1758",1,NaN,adult,hane,...,NaN,NaN,NaN,NaN,NaN,Anders Eriksson,Anders Eriksson,Anders Eriksson,NaN,NaN
17364574,55023,Ovaliderad,NaN,Ringtrast,Turdus torquatus,"Linnaeus, 1758",1,NaN,3K+,hane,...,NaN,NaN,NaN,NaN,NaN,Från Rixlarm,Från Rixlarm,NaN,NaN,NaN


## Data cleaning

#### Drop columns that are not needed

In [5]:
data = merged_dfs[["Antal","Ålder/stadium","Kön","Aktivitet","Lokalnamn","Ostkoordinat","Nordkoordinat","Noggrannhet","Diffusion","Län","Kommun","Provins","Startdatum","Starttid","Slutdatum","Sluttid"]]

In [111]:
#Let's check the result
print(data.head(4))
print("\n",len(data))

         observed_individuals    age   sex activity        place_name  \
Id                                                                      
17364564                    1  adult  hane      NaN       Fågelsundet   
17364574                    1    3K+  hane      NaN       Fågelsundet   
17370955                    1    NaN   NaN      NaN       Kungsgården   
17376315                    1  adult  hane      NaN  Österviks kapell   

          longitude  latitude  precision  diffusion    county  municipality  \
Id                                                                            
17364564    1616790   6722351        180          0   Uppsala         Tierp   
17364574    1616790   6722351        180          0   Uppsala         Tierp   
17370955    1537626   6234110        356          0    Kalmar    Mörbylånga   
17376315    1401480   6582480        179          0  Värmland  Kristinehamn   

          province  start_date start_time    end_date end_time  
Id                   

In [8]:
#Rename the columns to make them a bit easier to work with 
colnames = {"Antal":"observed_individuals",
"Ålder/stadium":"age",
"Kön":"sex",
"Aktivitet":"activity",
"Lokalnamn":"place_name",
"Ostkoordinat":"longitude",
"Nordkoordinat":"latitude",
"Noggrannhet":"precision",
"Diffusion":"diffusion",
"Län":"county",
"Kommun":"municipality",
"Provins":"province",
"Startdatum":"start_date",
"Starttid":"start_time",
"Slutdatum":"end_date",
"Sluttid":"end_time"}

data = data.rename(columns = colnames, inplace= False)
data.head(4)


,observed_individuals,age,sex,activity,place_name,longitude,latitude,precision,diffusion,county,municipality,province,start_date,start_time,end_date,end_time
Id,,,,,,,,,,,,,,,,
17364564,1,adult,hane,NaN,Fågelsundet,1616790,6722351,180,0,Uppsala,Tierp,Uppland,2000-04-20,11:32,2000-04-20,11:32
17364574,1,3K+,hane,NaN,Fågelsundet,1616790,6722351,180,0,Uppsala,Tierp,Uppland,2000-04-30,09:10,2000-04-30,09:10
17370955,1,NaN,NaN,NaN,Kungsgården,1537626,6234110,356,0,Kalmar,Mörbylånga,Öland,2000-04-18,13:28,2000-04-18,13:28
17376315,1,adult,hane,NaN,Österviks kapell,1401480,6582480,179,0,Värmland,Kristinehamn,Värmland,2000-04-25,19:59,2000-04-25,19:59


#### Check the data types and change if needed

In [23]:
data.dtypes

observed_individuals    object
age                     object
sex                     object
activity                object
place_name              object
longitude                int64
latitude                 int64
precision                int64
diffusion                int64
county                  object
municipality            object
province                object
start_date              object
start_time              object
end_date                object
end_time                object
dtype: object

#### Data type issues: 
* observed_individuals should be int but since artportalen allows you to report the number of reported individuals as "noterad" or "Ej återfunnen" python has interpreted this field as a string

* start_date, end date should be in date format
* start_time, end_time should be timestamp format

In [17]:
#Check how many of the observations are of the string type
not_found = data['observed_individuals'][data.observed_individuals == "Ej återfunnen"].count()
noted = data['observed_individuals'][data.observed_individuals == "noterad"].count()
total_missing = not_found + noted
print("There are {} observations with value 'Ej återfunnen', and {} observations with value 'noterad'.\nIn total {} (~ {} %) rows are missing numerical values in observed individuals.".format(not_found,noted,total_missing,(round((total_missing/40108)*100,0))))

There are 0 observations with value 'Ej återfunnen', and 0 observations with value 'noterad'.
In total 0 (~ 0.0 %) rows are missing numerical values in observed individuals.


#### Replace non numeric values in observed_individuals 
We can assume that there was at least one individual bird present for the observer to note the observation with "noterad" and that there where no birds present for the observer to note the observation as "Ej återfunnen". Therefore we make the follwing replacements:
* "noterad" -> 1
* "Ej återfunnen" -> 0 

In [15]:
data['observed_individuals'] = data['observed_individuals'].replace({"noterad":1,"Ej återfunnen":0})

#### Fix dates and insert ISO week number and month for the observations

In [10]:
#Set the dte and timestamp columns to correct date type

data['start_date'] = pd.to_datetime(data['start_date'])
data['end_date'] = pd.to_datetime(data['end_date'])



In [13]:
#Get the ISO week number from the start date, i.e. which week was the observation?
#Save result in column 'weeknumber'

def ISO_weeknumber(start_date):
    date_touple = date.isocalendar(start_date)
    return date_touple[1] 

data['weeknumber'] = data['start_date'].map(ISO_weeknumber)
data['month'] = pd.DatetimeIndex(data['start_date']).month



#### Save results to .csv

In [18]:
data.to_csv("torquatus.csv", sep=",", encoding = "UTF-8")